In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.output_parsers import (
    PydanticOutputParser,
    CommaSeparatedListOutputParser,
)
from pydantic import BaseModel, Field

chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.4)

with open("prompts/advisors.md", "r") as f:
    advisors_prompt = f.read()


advisors_parser = CommaSeparatedListOutputParser()

advisors_prompt_template = PromptTemplate(
    template=advisors_prompt,
    input_variables=["background", "problem"],
    partial_variables={
        "format_instructions": advisors_parser.get_format_instructions()
    },
)

background = "I'm a socially conscious AI developer, focusing on making AI that is smarter and more intelligent that humans, and can do the work that humans do more effectively."
problem = "Should I move to South boston?"

prompt = advisors_prompt_template.format(background=background, problem=problem)
personas = chat_model.predict(prompt)
personas = advisors_parser.parse(personas)

print(personas)

['Real Estate Agent', 'Career Advisor', 'Family Member', 'Local Resident', 'Financial Advisor']


In [8]:
import asyncio
from langchain.schema import AIMessage, HumanMessage, SystemMessage

with open("prompts/questions.md", "r") as f:
    questions_prompt = f.read()


class Category(BaseModel):
    category: str = Field(
        description="A category to be considered in relation to the problem"
    )
    questions_in_favour: list[str] = Field(
        description="Quesions to consider in favor of a course of action."
    )
    questions_against: list[str] = Field(
        description="Quesions to consider against of a course of action."
    )


class Categories(BaseModel):
    category: list[Category] = Field(
        description="Category to consider and questions in favor and against course of action."
    )


questions_parser = PydanticOutputParser(pydantic_object=Categories)

questions_prompt_template = PromptTemplate(
    template=questions_prompt,
    input_variables=["background", "problem", "advisor"],
    partial_variables={
        "format_instructions": questions_parser.get_format_instructions()
    },
)


async def async_prompt(chat_model, persona):
    prompt = questions_prompt_template.format(
        background=background, problem=problem, advisor=persona
    )
    c = await chat_model.agenerate([[HumanMessage(content=prompt)]])
    print(persona)
    return c


async def generate_concurrently(chat_model, personas):
    tasks = [async_prompt(chat_model, p) for p in personas]
    return await asyncio.gather(*tasks)


questions = await generate_concurrently(chat_model=chat_model, personas=personas)
questions = [
    questions_parser.parse(q.generations[0][0].text).model_dump() for q in questions
]

for i in range(len(personas)):
    questions[i]["advisor"] = personas[i]

Community Leader
Financial Advisor
Local Resident
Family Member
Environmental Activist
Career Mentor
Real Estate Agent


In [9]:
questions

[{'description': 'Why this perspective is relevant to my decision.',
  'category': [{'category': 'Location',
    'questions_in_favour': ['Is South Boston a safe neighborhood?',
     'Are there good schools in South Boston?',
     'Is South Boston close to my workplace?',
     'Are there parks and recreational facilities in South Boston?',
     'Are there amenities such as grocery stores, restaurants, and shopping centers nearby?'],
    'questions_against': ['Are there high crime rates in South Boston?',
     'Are the schools in South Boston of low quality?',
     'Is South Boston far from my workplace?',
     'Are there limited green spaces and recreational facilities in South Boston?',
     'Are there limited amenities in South Boston?']},
   {'category': 'Housing Market',
    'questions_in_favour': ['Is the housing market in South Boston stable and growing?',
     'Are there affordable housing options in South Boston?',
     'Is there a high demand for housing in South Boston?',
    

What do I need to find out to answer this question?
